In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [7]:

class Constants():
    ''' The Constants for the mapping terms to index '''
    PAD = 0
    UNK = 1
    SOS = 2
    EOS = 3

    PAD_WORD = '<PAD>'
    UNK_WORD = '<UNK>'
    SOS_WORD = '<SOS>'
    EOS_WORD = '<EOS>'

In [8]:

class BaselineLSTMModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int,  lstm_hidden: int, droup_out: float, num_lstm_layers: int):
        super(BaselineLSTMModel, self).__init__()

        self.emb = nn.Embedding(vocab_size, embedding_dim, Constants.PAD)
        self.lstm_1 = nn.LSTM(embedding_dim, lstm_hidden, batch_first=True, dropout= dropout, num_layers= num_lstm_layers)

        self.output_net = nn.Sequential(
            nn.Linear(lstm_hidden, vocab_size),
            nn.Softmax()
        )

        ## The trainable init h0 and c0 in LSTM.
        self.h0 = nn.Parameter(torch.randn(1, lstm_hidden))
        self.c0 = nn.Parameter(torch.randn(1, lstm_hidden))

In [17]:
emb_dim: int  = 256
batch_size = 16
vocab_size = 40
seq_len = 20


fake_data = torch.randint(0, 10, (batch_size, seq_len))

In [34]:
fake_data

tensor([[9, 3, 8, 1, 6, 0, 6, 4, 6, 6, 5, 5, 0, 4, 2, 1, 1, 9, 5, 2],
        [0, 0, 7, 3, 1, 8, 6, 2, 5, 7, 3, 7, 6, 0, 9, 1, 6, 4, 6, 7],
        [2, 6, 5, 4, 8, 9, 2, 5, 9, 0, 5, 0, 3, 2, 2, 5, 5, 8, 9, 5],
        [0, 9, 5, 5, 8, 1, 6, 1, 9, 0, 8, 0, 6, 0, 3, 1, 5, 2, 7, 1],
        [5, 6, 0, 2, 0, 2, 8, 6, 5, 1, 6, 4, 1, 9, 6, 9, 4, 7, 5, 4],
        [9, 8, 5, 4, 1, 0, 2, 8, 3, 7, 4, 5, 8, 1, 9, 5, 5, 8, 9, 2],
        [0, 6, 3, 9, 7, 7, 8, 3, 1, 9, 9, 9, 9, 2, 9, 2, 3, 4, 0, 1],
        [7, 2, 6, 3, 1, 3, 0, 6, 3, 6, 3, 3, 1, 8, 8, 0, 1, 2, 1, 7],
        [4, 0, 1, 4, 2, 2, 6, 0, 5, 7, 2, 0, 6, 7, 1, 5, 8, 0, 7, 4],
        [7, 4, 7, 7, 8, 6, 6, 4, 4, 1, 1, 4, 8, 6, 8, 4, 9, 7, 7, 2],
        [5, 1, 7, 9, 2, 6, 6, 1, 7, 4, 2, 8, 0, 8, 8, 6, 7, 1, 3, 3],
        [0, 7, 8, 5, 5, 6, 8, 1, 8, 1, 0, 8, 7, 8, 5, 2, 7, 8, 1, 6],
        [7, 8, 2, 5, 1, 8, 4, 7, 0, 8, 1, 5, 2, 6, 4, 0, 3, 0, 9, 6],
        [9, 3, 4, 1, 2, 6, 1, 2, 2, 1, 3, 9, 8, 8, 3, 6, 6, 2, 6, 3],
        [7, 2, 5, 2,

In [37]:
lstm_hidden

(tensor([[[-0.5815,  0.2030, -0.0202,  ...,  0.8039, -0.5579,  0.0098],
          [ 0.4073, -0.0496, -0.1913,  ..., -0.0657,  0.7767, -0.3504],
          [-0.2486, -0.0098,  0.3298,  ...,  0.0799, -0.4597, -0.0956],
          ...,
          [-0.0109,  0.0360, -0.6845,  ..., -0.0649, -0.0043,  0.2336],
          [-0.2057,  0.1425,  0.0550,  ...,  0.0782, -0.2461,  0.0945],
          [-0.1666,  0.1623,  0.1167,  ...,  0.0782, -0.4302,  0.0701]],
 
         [[ 0.1281,  0.0067, -0.0837,  ..., -0.1079,  0.0955,  0.1091],
          [-0.0275,  0.0645,  0.0321,  ..., -0.1205,  0.0091,  0.1253],
          [ 0.0012,  0.0579,  0.0402,  ...,  0.0322, -0.0506,  0.0794],
          ...,
          [ 0.0788,  0.0189, -0.0584,  ..., -0.0695,  0.2621,  0.0312],
          [-0.0230,  0.0535,  0.1314,  ..., -0.0962,  0.1807,  0.1278],
          [ 0.0287, -0.0373,  0.0254,  ...,  0.0532,  0.0538,  0.1610]]],
        grad_fn=<StackBackward>),
 tensor([[[-0.7333,  0.2697, -0.0363,  ...,  1.8080, -1.3395,  0.10

In [51]:
lstm_layer=2 

In [74]:
h0 = nn.Parameter(torch.randn(lstm_layer,1, lstm_hidden))
c0 = nn.Parameter(torch.randn(lstm_layer,1, lstm_hidden))

In [75]:
h0.shape

torch.Size([2, 1, 32])

In [76]:
emb_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim= emb_dim)

In [77]:
emb_out = emb_layer(fake_data)

In [78]:
emb_out.shape

torch.Size([16, 20, 256])

In [79]:
lstm_hidden = 32

In [80]:
lstm = nn.LSTM(emb_dim, lstm_hidden, batch_first=True, dropout= 0.8, num_layers= 2)

In [81]:
h0.shape

torch.Size([2, 1, 32])

In [82]:
lstm_out, lstm_h =  lstm(emb_out, (h0.repeat(1, batch_size, 1) ,c0.repeat(1, batch_size, 1)))

In [83]:
lstm_out.shape

torch.Size([16, 20, 32])

In [85]:
lstm_out[:, -1, :].shape

torch.Size([16, 32])

In [86]:
output_net = nn.Linear(lstm_hidden, vocab_size)

In [88]:
out = output_net(lstm_out)

In [89]:
out.shape

torch.Size([16, 20, 40])

In [99]:
lossCal = nn.CrossEntropyLoss( )

In [110]:
final_loss = lossCal(out[:, :-1,:].permute(0, 2,1 ), fake_data[:, 1:])

In [111]:
final_loss

tensor(3.7325, grad_fn=<NllLoss2DBackward>)

In [112]:
final_loss.backward()

In [107]:
fake_data[:, 1:].shape

torch.Size([16, 19])

In [106]:
seq_len

20